In [121]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [122]:
# есть как минимум пара фильмов (tt1966566, tt0402022), где с кодировкой проблемы.
# должна по идее подойти CP1252, но ругается, оставил utf-8
data = pd.read_csv('movie_bd_v5.xls', encoding='utf-8')
data.sort_values('original_title', ascending=False).head()

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1311,tt1966566,30000000,68129518,Ð¡Ñ‚Ð°Ð»Ð¸Ð½Ð³Ñ€Ð°Ð´,Thomas Kretschmann|Yanina Studilina|Philippe R...,Fyodor Bondarchuk,The Epic Battle That Turned The Tide Of World ...,"Drama set in 1942, during one of the most impo...",131,War|Action,Art Pictures Studio,10/9/2013,5.5,2013
1325,tt0402022,62000000,52304001,Ã†on Flux,Charlize Theron|Marton Csokas|Jonny Lee Miller...,Karyn Kusama,The Perfect World Meets The Perfect Assassin,"400 years into the future, disease has wiped o...",93,Action|Science Fiction,Paramount Pictures|Lakeshore Entertainment|MTV...,11/30/2005,5.3,2005
1342,tt0329774,87000000,71073932,xXx: State of the Union,Ice Cube|Willem Dafoe|Samuel L. Jackson|Scott ...,Lee Tamahori,Prepare for the next level,"Ice Cube stars as Darius Stone, a thrill-seeki...",101,Action|Crime|Adventure,Original Film|Columbia Pictures Corporation|Re...,4/27/2005,4.8,2005
876,tt0295701,70000000,277448382,xXx,Vin Diesel|Asia Argento|Marton Csokas|Samuel L...,Rob Cohen,A New Breed Of Secret Agent.,Xander Cage is your standard adrenaline junkie...,124,Action,Columbia Pictures|Original Film|Revolution Stu...,8/9/2002,5.7,2002
1529,tt0383060,35000000,12506188,Zoom,Tim Allen|Courteney Cox|Chevy Chase|Spencer Br...,Peter Hewitt,They're going to save the world... as long as ...,Jack Shepard is an out-of-shape auto shop owne...,83,Fantasy|Comedy|Family,Columbia Pictures|Revolution Studios,8/11/2006,5.1,2006


In [123]:
# Смотрим общие характеристики набора данных
data.describe()

,budget,revenue,runtime,vote_average,release_year
count,1.889000e+03,1.889000e+03,1889.000000,1889.000000,1889.000000
mean,5.431083e+07,1.553653e+08,109.658549,6.140762,2007.860773
std,4.858721e+07,2.146698e+08,18.017041,0.764763,4.468841
min,5.000000e+06,2.033165e+06,63.000000,3.300000,2000.000000
25%,2.000000e+07,3.456058e+07,97.000000,5.600000,2004.000000
50%,3.800000e+07,8.361541e+07,107.000000,6.100000,2008.000000
75%,7.200000e+07,1.782626e+08,120.000000,6.600000,2012.000000
max,3.800000e+08,2.781506e+09,214.000000,8.100000,2015.000000


# Предобработка

In [124]:
answers = {} # создадим словарь для ответов

# создадим колонку "Profit", она пригодится в некоторых вопросах
data['profit'] = data['revenue'] - data['budget']

# создадим колонку "Title Length" (нужна в 24-ом вопросе)
data['title length'] = data.original_title.apply(lambda x: len(x))

# создадим колонку "Overview Length" (нужна в 25-ом вопросе)
data['overview length'] = data.overview.apply(lambda x: len(str(x).split()))

# создадим колонку "Month", где будет храниться месяц выхода фильма
data['month'] = data.release_date.apply(lambda x: int(x[:x.find('/')]))

# составляем список всех возможных жанров фильмов
genres_str = str()
# сшиваем все строки с жанрами в одну
for line in data['genres']:
    genres_str = genres_str + "|" + str(line)
# переводим строку в список и оставляем уникальные значения
genres_list = [genre for genre in genres_str.split('|') if len(genre)]
genres_series = pd.Series(genres_list).unique()

# составляем список актёров и кол-во фильмов, в которых они снимались
actors_str = str()
# собираем всех актёров из всех фильмов в одну длинную строчку,
# разделяя их вертикальной чертой
for line in data['cast']:
    actors_str = actors_str + "|" + str(line)
# превращаем строчку в список list
actors_list = [actor for actor in actors_str.split('|') if len(actor)]
# переводим список в Series (готовим к переводу в DataFrame)
actors_series = pd.Series(actors_list)
# подсчитываем кол-во фильмов у каждого актёра и формируем DataFrame
actors_df = actors_series.value_counts().reset_index()
# даём понятные названия колонкам
actors_df.columns = ["Actor's Name", 'Amount of Movies']

# составляем список компаний по предыдущей логике
companies_str = str()
# всё одной строкой
for line in data['production_companies']:
    companies_str = companies_str + "|" + str(line)
# переводим строку в список и оставляем уникальные значения
companies_list = [co for co in companies_str.split('|') if len(co)]
companies_series = pd.Series(companies_list).unique()

# оцениваем, что получилось
print(genres_series[:3], actors_df[:3], companies_series[:3], sep='\n'*3)

['Action' 'Adventure' 'Science Fiction']


        Actor's Name  Amount of Movies
0  Samuel L. Jackson                27
1       Nicolas Cage                27
2     Robert De Niro                26


['Universal Studios' 'Amblin Entertainment' 'Legendary Pictures']


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [125]:
# в словарь вставляем номер вопроса и ответ на него

answers['1'] = '+   Pirates of the Caribbean: On Stranger Tides'

# если ответили верно, добравляем значок "+"

In [126]:
# тут пишем ваш код для решения данного вопроса:
data[data['budget'] == data['budget'].max()].original_title

723    Pirates of the Caribbean: On Stranger Tides
Name: original_title, dtype: object

In [127]:
# для проверки посмотрим топ 3, чтобы убедиться, что он такой один
data[['budget', 'original_title']].sort_values('budget', ascending=False).head(3)

,budget,original_title
723,380000000,Pirates of the Caribbean: On Stranger Tides
1669,300000000,Pirates of the Caribbean: At World's End
14,280000000,Avengers: Age of Ultron


# 2. Какой из фильмов самый длительный (в минутах)?

In [128]:
answers['2'] = '+   Gods and Generals'

In [129]:
# ответ
data[data['runtime'] == data['runtime'].max()].original_title

1157    Gods and Generals
Name: original_title, dtype: object

In [130]:
# для проверки посмотрим топ 3, чтобы убедиться, что он такой один
data[['original_title', 'runtime']].sort_values('runtime', ascending=False).head(3)

,original_title,runtime
1157,Gods and Generals,214
1081,The Lord of the Rings: The Return of the King,201
1736,Grindhouse,191


# 3. Какой из фильмов самый короткий (в минутах)?





In [131]:
answers['3'] = '+   Winnie the Pooh'

In [132]:
# ответ
data[data['runtime'] == data['runtime'].min()].original_title

768    Winnie the Pooh
Name: original_title, dtype: object

In [133]:
# для проверки посмотрим топ 3, чтобы убедиться, что он такой один
data[['original_title', 'runtime']].sort_values('runtime').head(3)

,original_title,runtime
768,Winnie the Pooh,63
930,Return to Never Land,72
1097,The Jungle Book 2,72


# 4. Какова средняя длительность фильмов?


In [134]:
answers['4'] = '+   110'

In [135]:
# ответ
data['runtime'].mean()

109.6585494970884

# 5. Каково медианное значение длительности фильмов? 

In [136]:
answers['5'] = '+   107'

In [137]:
# ответ
data['runtime'].median()

107.0

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [138]:
answers['6'] = '+   Avatar'

In [139]:
# ответ
data[data['profit'] == data['profit'].max()].original_title

239    Avatar
Name: original_title, dtype: object

In [140]:
# для проверки посмотрим топ 3, чтобы убедиться, что он такой один
data[['original_title', 'profit']].sort_values('profit', ascending=False).head(3)

,original_title,profit
239,Avatar,2544505847
3,Star Wars: The Force Awakens,1868178225
0,Jurassic World,1363528810


# 7. Какой фильм самый убыточный? 

In [141]:
answers['7'] = '+   The Lone Ranger'

In [142]:
# ответ
data[data['profit'] == data['profit'].min()].original_title

1245    The Lone Ranger
Name: original_title, dtype: object

In [143]:
# для проверки посмотрим топ 3, чтобы убедиться, что он такой один
data[['original_title', 'profit']].sort_values('profit').head(3)

,original_title,profit
1245,The Lone Ranger,-165710090
1616,The Alamo,-119180039
809,Mars Needs Moms,-111007242


# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [144]:
answers['8'] = '+   1478'

In [145]:
# ответ
data[data['revenue'] > data['budget']].imdb_id.count()

1478

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [146]:
answers['9'] = '+   The Dark Knight'

In [147]:
# ответ: посмотрим топ 3 по сборам за 2008 год
data[data['release_year'] == 2008].sort_values(['revenue'], ascending=False)[['original_title', 'revenue']].head(3)

,original_title,revenue
599,The Dark Knight,1001921825
603,Indiana Jones and the Kingdom of the Crystal S...,786636033
606,Kung Fu Panda,631744560


# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [148]:
answers['10'] = '+   The Lone Ranger'

In [149]:
# ответ: посмотрим топ 3 по убыткам за 2012-2014 года
data.query('2012 <= release_year <= 2014')[['original_title', 'profit']].sort_values('profit').head(3)

,original_title,profit
1245,The Lone Ranger,-165710090
1214,R.I.P.D.,-68351500
1007,Upside Down,-51893525


# 11. Какого жанра фильмов больше всего?

In [150]:
answers['11'] = '+   Drama'

ВАРИАНТ 1

In [151]:
# используем уже собранный ранее список genres_list
q11_version_a = pd.Series(genres_list).value_counts()

# ответ
q11_version_a

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64

ВАРИАНТ 2

In [152]:
# этот вариант заметно дольше
q11_version_b = pd.DataFrame()

# будем наполнять DataFrame строками из поля "genres", 
# преобразованными в Series
for line in data['genres']:
    temp = pd.Series(str(line).split('|'))
    q11_version_b = pd.concat([q11_version_b, temp])

# ответ
q11_version_b.value_counts()

Drama              782
Comedy             683
Thriller           596
Action             582
Adventure          415
Crime              315
Romance            308
Family             260
Science Fiction    248
Fantasy            222
Horror             176
Mystery            168
Animation          139
Music               64
History             62
War                 58
Western             19
Documentary          8
Foreign              2
dtype: int64


# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

In [153]:
answers['12'] = '+   Drama'

Хотел бы добавить комментарий. К этому вопросу можно подойти с 2х сторон, потому что не ясно, что значит "чаще". Чаще по частоте, или чаще в процентном соотношении...

In [154]:
# создадим список с общим количеством фильмов по жанру
q12_total = list()
# создадим список с количеством прибыльных фильмов по жанру
q12_profitable = list()
# наполним их обходом по датасету
for genre in genres_series:
    q12_total.append(data[data.genres.str.contains(genre)].imdb_id.count())
    q12_profitable.append(data[(data.genres.str.contains(genre)) & (data.profit > 0)].imdb_id.count())
# создадим словарь с данными
q12_dict = {
    'genres': genres_series,
    'profitable': q12_profitable,
    'total': q12_total
    }
# переведем данные в DataFrame
q12 = pd.DataFrame(q12_dict)
# оформим столбцы данных
q12['percentage'] = q12['profitable'] / q12['total'] * 100
q12['percentage'] = q12['percentage'].apply(lambda x: str(int(x)) + '%')
# ответ
q12.sort_values('percentage', ascending=False)

# из таблицы видно, что документальные фильмы имеют успех в 87%, а драма только в 71%
# с другой стороны драма встречается чаще.

,genres,profitable,total,percentage
17,Documentary,7,8,87%
9,Animation,120,139,86%
8,Family,226,260,86%
16,Horror,150,176,85%
4,Fantasy,188,222,84%
1,Adventure,337,415,81%
10,Comedy,551,683,80%
2,Science Fiction,195,248,78%
12,Romance,242,308,78%
0,Action,444,582,76%


# 13. У какого режиссера самые большие суммарные кассовые сборы?

In [155]:
answers['13'] = '+   Peter Jackson'

In [156]:
# ответ
data.groupby('director').revenue.sum().sort_values(ascending=False).head(3)

director
Peter Jackson        6490593685
Christopher Nolan    4167548502
David Yates          4154295625
Name: revenue, dtype: int64

# 14. Какой режисер снял больше всего фильмов в стиле Action?

Комментарий. Получаются разные "победители", если использовать str.match и str.contains. См. ниже

P.S. я ответил правильно только со второго раза, так как считал по методу str.contains

In [157]:
answers['14'] = '+   Robert Rodriguez'

In [158]:
# ответ
data[data.genres.str.match('Action')].director.value_counts().head()

Andrzej Bartkowiak    5
Antoine Fuqua         5
Robert Rodriguez      5
Tony Scott            5
Zack Snyder           4
Name: director, dtype: int64

In [159]:
# ответ
data[data.genres.str.contains('Action')].director.value_counts().head()

Michael Bay           7
Paul W.S. Anderson    7
Robert Rodriguez      6
Ridley Scott          6
Antoine Fuqua         6
Name: director, dtype: int64

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

In [160]:
answers['15'] = '+   Chris Hemsworth'

In [161]:
q15_list = list()
# наполняем список по каждому актёру за 2012 год по сумме его сборов
for actor in actors_df["Actor's Name"]:
    q15_list.append(data[(data.cast.str.contains(actor)) & (data.release_year == 2012)].revenue.sum())

# создаём словарь, который переведём в DataFrame
q15_dict = {
    'actors': actors_df["Actor's Name"],
    'revenue 2012': q15_list
    }
q15 = pd.DataFrame(q15_dict)

# ответ
q15.sort_values('revenue 2012', ascending=False).head()

,actors,revenue 2012
104,Chris Hemsworth,2027450773
591,Denis Leary,1629460639
25,Anne Hathaway,1522851057
59,Scarlett Johansson,1519557910
54,Robert Downey Jr.,1519557910


# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

In [162]:
answers['16'] = '+   Matt Damon'

In [163]:
q16_qty_movies = list()
q16_actors = list()
# здесь будем хранить максимальное число высокобюджетных фильмов
max_num = int()
for actor in actors_df["Actor's Name"]:
    # условие, которое увеличило скорость в 51 раз
    # так как актёры в списке упорядочены по убыванию кол-ва фильмов,
    # то, как только общее кол-во фильмов очередного актёра, меньше,
    # чем кол-во уже найденных высокобюджетных фильмов, то цикл прерывается
    if int(actors_df[actors_df["Actor's Name"] == actor]['Amount of Movies']) < max_num:
        break
    # результат поиска
    qty = data[(data.budget > data.budget.mean()) & data.cast.str.contains(actor)].imdb_id.count()
    # заносим в списки
    q16_qty_movies.append(qty)
    q16_actors.append(actor)
    # обновляем значение максимального числа если необходимо
    max_num = qty if qty > max_num else max_num

# создаем словарь с полученными данными и переводим его в DataFrame
most_act = {
    'actor': q16_actors,
    'times': q16_qty_movies
    }
q16 = pd.DataFrame(most_act)

# ответ
q16.sort_values('times', ascending=False).head()

,actor,times
3,Matt Damon,18
14,Adam Sandler,17
18,Angelina Jolie,16
0,Samuel L. Jackson,15
23,Hugh Jackman,14


# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

In [164]:
answers['17'] = '+   Action'

In [165]:
# в список будем добавлять кол-во раз для каждого жанра
q17_list = list()
for genre in genres_series:
    q17_list.append(data[(data.cast.str.contains('Nicolas Cage')) & data.genres.str.contains(genre)].imdb_id.count())

# из данных создаем словарь и переводим его в DataFrame
q17_dict = {
    'genres': genres_series,
    'times': q17_list
    }
q17 = pd.DataFrame(q17_dict)

# ответ
q17.sort_values('times', ascending=False)

,genres,times
0,Action,17
3,Thriller,15
7,Drama,12
5,Crime,10
4,Fantasy,8
1,Adventure,7
10,Comedy,6
2,Science Fiction,4
11,Mystery,3
9,Animation,3


# 18. Самый убыточный фильм от Paramount Pictures

In [166]:
answers['18'] = '+   K-19: The Widowmaker'

In [167]:
# ответ: посмотрим топ 3 убыточных фильмов от Paramount Pictures
data[data.production_companies.str.contains('Paramount Pictures')][['production_companies', 'original_title', 'profit']].sort_values('profit').head(3)

,production_companies,original_title,profit
925,Paramount Pictures|Intermedia Films|National G...,K-19: The Widowmaker,-64831034
1123,Paramount Pictures|Donners' Company|Mutual Fil...,Timeline,-60519261
1722,Paramount Pictures|Virtual Studios|Revolution ...,Next,-51788987


# 19. Какой год стал самым успешным по суммарным кассовым сборам?

In [168]:
answers['19'] = '+   2015'

In [169]:
# ответ
data.groupby('release_year').revenue.sum().sort_values(ascending=False)

release_year
2015    25449202382
2014    23405862953
2013    23213799791
2012    23079001687
2011    22676791872
2010    21071204961
2009    20261791024
2008    18252781990
2007    18162406801
2004    15663430720
2005    15309425558
2006    14775042320
2003    14346123312
2002    14136361487
2001    13017764865
2000    10664099805
Name: revenue, dtype: int64

# 20. Какой самый прибыльный год для студии Warner Bros?

In [170]:
answers['20'] = '+   2014'

In [171]:
# группируем по годам и применяем функцию sum()
q20_profits = data[data.production_companies.str.contains('Warner Bros')].groupby('release_year').profit.sum()
# ответ сортируем
q20_profits.sort_values(axis=0, ascending=False)

release_year
2014    2295464519
2007    2201675217
2008    2134595031
2010    1974712985
2011    1871393682
2003    1855493377
2009    1822454136
2013    1636453400
2004    1631933725
2005    1551980298
2001    1343545668
2012    1258020056
2002    1022709901
2015     870368348
2006     620170743
2000     452631386
Name: profit, dtype: int64

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

In [172]:
answers['21'] = '+   9-сентябрь'

In [173]:
# ответ путём группировки по месяцам
data.groupby('month').imdb_id.count().sort_values(ascending=False)

month
9     227
12    190
10    186
8     161
3     156
4     149
6     147
11    146
7     142
5     140
2     135
1     110
Name: imdb_id, dtype: int64

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

In [174]:
answers['22'] = '+   450'

In [175]:
# ответ в лоб через использования query()
data.query('6 <= month <= 8').imdb_id.count()

450

# 23. Для какого режиссера зима – самое продуктивное время года? 

In [176]:
answers['23'] = '+   Peter Jackson'

In [177]:
# ответ с группировкой по режиссёрам
data.query('month in [1, 2, 12]').director.value_counts()

Peter Jackson        7
Steven Soderbergh    6
Clint Eastwood       6
Shawn Levy           4
Adam Shankman        4
                    ..
Jeremy Garelick      1
Bill Condon          1
Peter Howitt         1
Matt Reeves          1
Tom Hooper           1
Name: director, Length: 332, dtype: int64

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

In [178]:
answers['24'] = '+   Four By Two Productions'

In [181]:
# выдаётся предупреждение из-за того, что некоторые фильмы содержат апострофы.
# это решаемо через махинации с re.

# список для хранения средних длин названий фильмов
q24_lengths = list()
# в цикле перебираем все компании из списка
for company in companies_series:
    mean_length = data[data.production_companies.str.contains(company)]['title length'].mean()
    q24_lengths.append(mean_length)
    
# составляем словарь, а на его основе DataFrame
mean_lengths = {
    'company': companies_series,
    'mean length': q24_lengths
    }
q24 = pd.DataFrame(mean_lengths)

# ответ
q24.sort_values('mean length', ascending=False).head()

/home/pavlk/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,company,mean length
1505,Four By Two Productions,83.0
389,"Jim Henson Company, The",59.0
1211,Dos Corazones,47.0
450,Museum Canada Productions,46.0
547,Polsky Films,46.0


# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

In [182]:
answers['25'] = '+   Midnight Picture Show'

In [185]:
# выдаётся предупреждение из-за того, что некоторые фильмы содержат апострофы.
# это решаемо через махинации с re.

# список для хранения средних длин описаний фильмов
q25_words = list()
# в цикле перебираем все компании из списка
for company in companies_series: 
    mean_words = data[data.production_companies.str.contains(company)]['overview length'].mean()
    q25_words.append(mean_words)
    
# составляем словарь, а на его основе DataFrame
q25_dict = {
    'company': companies_series,
    'mean words': q25_words
    }
res_over = pd.DataFrame(q25_dict)

# ответ
res_over.sort_values('mean words', ascending=False).head()

/home/pavlk/anaconda3/lib/python3.8/site-packages/pandas/core/strings.py:2001: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


,company,mean words
950,Midnight Picture Show,175.0
1445,Room 9 Entertainment,161.0
1141,98 MPH Productions,159.0
1232,Heineken Branded Entertainment,159.0
1039,Brookwell-McNamara Entertainment,156.0


# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

In [186]:
answers['26'] = '+   The Dark Knight/Inside Out/12 Years a Slave'

In [187]:
# ответ через использования quantile, чтобы задать топ 1 процент по рейтингу
data[data['vote_average'] > data['vote_average'].quantile(0.99)][['original_title', 'vote_average']].sort_values('vote_average', ascending=False)

,original_title,vote_average
599,The Dark Knight,8.1
9,Inside Out,8.0
34,Room,8.0
118,Interstellar,8.0
125,The Imitation Game,8.0
119,Guardians of the Galaxy,7.9
128,Gone Girl,7.9
138,The Grand Budapest Hotel,7.9
370,Inception,7.9
872,The Pianist,7.9


# 27. Какие актеры чаще всего снимаются в одном фильме вместе?

Комментарий: на моём компьютере это занимает ровно 3 минуты

In [188]:
answers['27'] = "+   Daniel Radcliffe и Emma Watson 8 раз"

In [189]:
# список для сохранения пар актёров
q27_actors = list()
# список для сохранения кол-ва их совместных работ
q27_mutual = list()
# максимальное значение пересечений актёров
max_crossed = int()

# начнём цикл перебора актёров, помним, они у нас упорядочены по количеству фильмов
# enumerate поможет нам брать только последующих актёров из второго списка, чтобы избежать двойной работы
for num, actor1 in enumerate(actors_df["Actor's Name"]):
    # вылетаем из цикла, если у очередного актёра фильмов меньше, чем уже найденное максимальное кол-во пересечений
    if int(actors_df[actors_df["Actor's Name"] == actor1]['Amount of Movies']) < max_crossed:
        break
    # начинаем перебор с актёра ступенькой ниже
    for actor2 in actors_df["Actor's Name"][num + 1:]:
        # вылетаем из цикла, если у очередного актёра фильмов меньше, чем уже найденное максимальное кол-во пересечений
        if int(actors_df[actors_df["Actor's Name"] == actor2]['Amount of Movies']) < max_crossed:
            break
        # кол-во совместных фильмов
        times_crossed = data[(data.cast.str.contains(actor1)) & (data.cast.str.contains(actor2))].imdb_id.count()
        # делаем запись данных в списки, если необходимо
        if times_crossed and times_crossed >= max_crossed:
            max_crossed = times_crossed
            q27_actors.append(f'{actor1} & {actor2}')
            q27_mutual.append(times_crossed)

# формируем словарь с данными, а на его основе DataFrame
q27_dict = {
    'actors': q27_actors,
    'times crossed': q27_mutual
    }
q27 = pd.DataFrame(q27_dict)

# ответ
q27.sort_values('times crossed', ascending=False).head()

,actors,times crossed
12,Emma Watson & Daniel Radcliffe,8
13,Emma Watson & Rupert Grint,8
14,Daniel Radcliffe & Rupert Grint,8
10,Owen Wilson & Ben Stiller,6
11,Johnny Depp & Helena Bonham Carter,6


# Submission

In [190]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': '+   Pirates of the Caribbean: On Stranger Tides',
 '2': '+   Gods and Generals',
 '3': '+   Winnie the Pooh',
 '4': '+   110',
 '5': '+   107',
 '6': '+   Avatar',
 '7': '+   The Lone Ranger',
 '8': '+   1478',
 '9': '+   The Dark Knight',
 '10': '+   The Lone Ranger',
 '11': '+   Drama',
 '12': '+   Drama',
 '13': '+   Peter Jackson',
 '14': '+   Robert Rodriguez',
 '15': '+   Chris Hemsworth',
 '16': '+   Matt Damon',
 '17': '+   Action',
 '18': '+   K-19: The Widowmaker',
 '19': '+   2015',
 '20': '+   2014',
 '21': '+   9-сентябрь',
 '22': '+   450',
 '23': '+   Peter Jackson',
 '24': '+   Four By Two Productions',
 '25': '+   Midnight Picture Show',
 '26': '+   The Dark Knight/Inside Out/12 Years a Slave',
 '27': '+   Daniel Radcliffe и Emma Watson 8 раз'}

In [191]:
# и убедиться что ничего не пропустил
len(answers)

27